In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI 
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory (
    llm=llm, 
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,       # 출력형식을 단순문자열 방식 말고 대화형식으로    
)

# 단순문자열 형식의 템플릿
# template = """
#     You are a helpful AI talking to a human.
#     {chat_history}
#     Human:{question}
#     You:
# """

# 대화형식의 템플릿
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name="chat_history"),          # 누가 보냈는지 알 수 없는, 예측하기 어려운 메세지의 양과 제한 없는 양의 메세지를 가질수 있음.
    ("human", "{question}")
])

def load_memory(_):
    #print(input)
    return memory.load_memory_variables({})["chat_history"]

chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question":"My name is Nico"})
    memory.save_context({"input": question}, {"outputs":result.content},)
    print(result)



AIMessage(content='Hello Nico! How can I assist you today?')